In [ ]:
import numpy as np
import tritonclient.grpc as grpcclient

In [ ]:
TRITON_SERVER_URL = "172.25.4.42:8001"
MODEL_NAME = "bert-base-uncased"
MODEL_VERSION = "1"
INPUT__0 = (
    np.array(list(bytes("Who were the founders of NVIDIA ?", "utf8")))
    .reshape(1, -1)
    .astype("uint8")
)
INPUT__1 = (
    np.array(
        list(
            bytes(
                "NVIDIA was founded by Jensen Huang, Chris Malachowsky, and Curtis Priem in 1993.",
                "utf8",
            )
        )
    )
    .reshape(1, -1)
    .astype("uint8")
)

In [ ]:
triton_client = grpcclient.InferenceServerClient(url=TRITON_SERVER_URL, verbose=False)

model_metadata = triton_client.get_model_metadata(
    model_name=MODEL_NAME, model_version=MODEL_VERSION
)

model_config = triton_client.get_model_config(
    model_name=MODEL_NAME, model_version=MODEL_VERSION
).config

In [ ]:
inputs = [
    grpcclient.InferInput("INPUT__0", INPUT__0.shape, "UINT8"),
    grpcclient.InferInput("INPUT__1", INPUT__1.shape, "UINT8"),
]
inputs[0].set_data_from_numpy(INPUT__0)
inputs[1].set_data_from_numpy(INPUT__1)

outputs = [grpcclient.InferRequestedOutput("OUTPUT__0")]

response = triton_client.infer(MODEL_NAME, inputs, outputs=outputs)

In [ ]:
response.get_response()